# Neural Network Model to Transform Response Vectors into Image Feature Vectors

In [46]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

## Build network

In [138]:
# build neural network to predict image feature vectors based on response vectors
class ImageFeaturePredict:
    
    def __init__(self,resp_vectors,image_feature_vectors):
        
        self.resp = resp_vectors # shape of (nCells,nFrames)
        self.feat = image_feature_vectors # shape of (nFrames,nWavelets)
    
    class Network(nn.Module):
        
        def __init__(self,n_cells,n_wavelets,hidden_dim_list):
            '''initialize constructed model with input layer, output layer, 
            and len(hidden_dim_list) hidden layers
            input_layer size is number of cells in response vector
            output_layer size is number of image features in feature vector'''
        
            # calling the constructor of the parent class - gets everything we need from pytorch
            super(ImageFeaturePredict.Network, self).__init__()
            
            ## Define layers of network
            # first layer input: # of cells in a response_vector
            self.input_layer = nn.Linear(n_cells,hidden_dim_list[0])
            
            # add hidden layers of network based on list of dimensions for each hidden layer
            self.hidden_layers = [] * len(hidden_dim_list)
            for i in range(len(hidden_dim_list) - 1):
                self.hidden_layers.append(nn.Linear(hidden_dim_list[i],hidden_dim_list[i+1]))
            
            # last layer output: # of Gabor wavelets/image features in a feature_vector
            self.output_layer = nn.Linear(hidden_dim_list[-1],n_wavelets)
            
            # non-linear activation function between layers
            self.tanh = nn.Tanh() # use tanh to keep activations between -1 and 1
#             self.relu = nn.ReLU() 
        
        def forward(self,batch):
            '''define the forward steps in the model between the layers;
            data batches are passed between layers after computing the non-linear transformations 
            on the activations'''
            
            # input layer
            batch = self.input_layer(batch)
            batch = self.tanh(batch)
      
            # computer the activations for each hidden layer
            for h in self.hidden_layers:
                batch = h(batch)
                batch = self.tanh(batch)
            
            # output layer
            batch = self.output_layer(batch)
            batch = nn.functional.tanh(batch)
            
            return batch
        

    def train_test_network(self,test_size,batch_size,hidden_dim_list,learning_rate,n_epochs):
            
        ## organize data
        # split the data into a training and test set
        train_resp,test_resp,train_feat,test_feat = train_test_split(self.resp,self.feat,test_size=test_size,shuffle=True)
        
        # create data batches
        train_resp_batches,train_feat_batches = batch_data(train_resp,train_feat,batch_size=batch_size)
        
        ## organize model and learning conditions
        # construct network
        neural_network = ImageFeaturePredict.Network(len(train_resp[0]),len(train_feat[0]),hidden_dim_list)
        # define optimization function on network - using stochastic gradient descent (SGD)
        optimizer = optim.SGD(neural_network.parameters(),lr=learning_rate)
        # define loss function
        loss_function = nn.MSELoss()
#         loss_function = nn.CrossEntropyLoss()
                
        ## begin to train the network
        neural_network.train()
        
        # train model over specified number of epochs
        for iE in range(n_epochs):
            
            correlations = [] # keep track of performance over multiple batches of data
            
            # iterate over each data batch
            for iB in range(len(train_resp_batches)):
                
                # pull out data batch (resp=data batch) (feat=labels)
                resp = train_resp_batches[iB]
                feat = train_feat_batches[iB]
                
                # reset optimizer
                optimizer.zero_grad()
                
                # calculate model predictions on batch
                predictions = neural_network(torch.tensor(resp.astype(np.float32)))
                
                # calculate error of prediction
                loss = loss_function(predictions,torch.FloatTensor(feat))
                # backward calculation step
                loss.backward()
                
                # calculate and update weights of network
                optimizer.step()
                
                # test model predictions at this epoch using correlation coefficient as measure of accuracy
                for iF,prediction in enumerate(predictions.data):
                    real = feat[iF]
                    
#                     print(np.min(np.asarray(real)),np.max(np.asarray(real)))
#                     print(np.min(np.asarray(prediction)),np.max(np.asarray(prediction)))
                    
                    corr = pearsonr(real,np.asarray(prediction))
                    correlations.append(corr)
                        
            # report prediction performance       
            print('Average Correlation for Epoch # ' + str(iE) + ': ' + str(np.round(np.mean(correlations),2)) )
        
                    
        ## evaluate the final model performance using test data
        neural_network.eval()
        
        test_correlations = []
        test_predictions = neural_network(torch.tensor(test_resp.astype(np.float32)))
                    
        # test model predictions using correlation coefficient as measure of accuracy
        for iF,prediction in enumerate(predictions.data):
            real = test_feat[iF]

            corr = pearsonr(real,prediction)
            test_correlations.append(corr)
                    
        # report prediction performance       
        print('Average Correlation for Test Data: ' + str(np.round(np.mean(test_correlations),2)) )
           
        return neural_network
        

## Utility Functions
def batch_data(data,labels,batch_size=16):
    
    data_batches = []
    label_batches = []

    for n in range(0,len(data),batch_size):
        if n+batch_size < len(data):
            data_batches.append(data[n:n+batch_size])
            label_batches.append(labels[n:n+batch_size])

    if len(data)%batch_size > 0:
        data_batches.append(data[len(data)-(len(data)%batch_size):len(data)])
        label_batches.append(labels[len(data)-(len(data)%batch_size):len(data)])
        
    return data_batches,label_batches

## Import data to train model

In [104]:
# load .npz of response vectors
resp_dict = np.load('./data/response_vectors.npz')
resp = resp_dict['response_vectors']
resp = np.swapaxes(resp,0,1)
print(resp.shape)

# load .npz of feature vectors
feat_dict = np.load('./data/feature_vectors.npz')
feat = feat_dict['feature_vectors']
feat = np.swapaxes(feat,0,1)
print(feat.shape)

nCell = resp.shape[1]
nWavelet = feat.shape[1]
print(nCell,nWavelet)

(36000, 132)
(36000, 1248)
132 1248


## Pass data to model to train and test
- may need to split up batches in model so sampling across tilings are uniform?

In [147]:
# model training parameters
test_size = 0.1
batch_size = 8
n_hidden_layers = 5
n_epochs = 5
learning_rate = 1

dim_step = (nWavelet-nCell)//(n_hidden_layers+1)
hidden_dim_list = np.arange(nCell+dim_step,nWavelet,dim_step)
print(hidden_dim_list)

[ 318  504  690  876 1062]


In [148]:
testclass = ImageFeaturePredict(resp,feat)

model = testclass.train_test_network(test_size=test_size,
                                     batch_size=batch_size,
                                     hidden_dim_list=hidden_dim_list,
                                     learning_rate=learning_rate,
                                     n_epochs=n_epochs)

Average Correlation for Epoch # 0: 0.44
Average Correlation for Epoch # 1: 0.45
Average Correlation for Epoch # 2: 0.45
Average Correlation for Epoch # 3: 0.45
Average Correlation for Epoch # 4: 0.45
Average Correlation for Test Data: 0.45
